In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
DATA_DIR="data/cleaned/"

In [ ]:
# import plotly.io as pio
# pio.renderers.default = 'browser'

In [2]:
# india_states = json.load(open("data/india_state.geojson", "r"))
india_states = json.load(open("data/vis/states_india.geojson", "r"))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]

In [3]:
df = pd.read_csv(DATA_DIR+"latlong.csv")
df['lat'] = df['lat'].round(3)
df['long'] = df['long'].round(3)
coords=df.groupby(['lat','long']).size().reset_index(name="freq")
coords.head()

,lat,long,freq
0,7.030,93.790,4
1,7.944,93.575,1
2,8.000,93.500,6
3,8.088,77.542,57
4,8.118,77.281,122


In [4]:
state_codes = pd.read_csv(DATA_DIR+"../vis/admin1CodesASCII.txt", delimiter="\t", names=['state_name', 'state_name2', 'pincode'])
states = df['state_name'].value_counts().reset_index(name='freq')
states['index'] = states['index'].apply(lambda x: 'IN.%.2d'%x)
states = states.join(state_codes['state_name'], on='index')
states['state_name'] = states['state_name'].str.replace(" and ", " & ")

# Some discrepancies, require manual editing
states['id']=states['state_name'].map(state_id_map)
states.dropna(inplace=True)
states.head()

,index,freq,state_name,id
0,IN.36,77833,Uttar Pradesh,9.0
1,IN.35,66156,Madhya Pradesh,23.0
3,IN.16,29360,Maharashtra,27.0
4,IN.34,24651,Bihar,10.0
5,IN.21,12875,Odisha,21.0


In [5]:
from sklearn import preprocessing

states['scaled']=np.log10(states['freq'])
coords['scaled']=np.log10(coords['freq'])
coords['scaled']=preprocessing.minmax_scale(coords['scaled'], (0,1))
states.head()

,index,freq,state_name,id,scaled
0,IN.36,77833,Uttar Pradesh,9.0,4.891164
1,IN.35,66156,Madhya Pradesh,23.0,4.820569
3,IN.16,29360,Maharashtra,27.0,4.467756
4,IN.34,24651,Bihar,10.0,4.391835
5,IN.21,12875,Odisha,21.0,4.109747


In [6]:
states

,index,freq,state_name,id,scaled
0,IN.36,77833,Uttar Pradesh,9.0,4.891164
1,IN.35,66156,Madhya Pradesh,23.0,4.820569
3,IN.16,29360,Maharashtra,27.0,4.467756
4,IN.34,24651,Bihar,10.0,4.391835
5,IN.21,12875,Odisha,21.0,4.109747
6,IN.38,12753,Jharkhand,20.0,4.105612
7,IN.25,11876,Tamil Nadu,33.0,4.074670
8,IN.19,10286,Karnataka,29.0,4.012247
9,IN.28,9345,West Bengal,19.0,3.970579
10,IN.24,8817,Rajasthan,8.0,3.945321


## MAIN

In [ ]:
fig = px.choropleth_mapbox(
    states,
    locations="id",
    geojson=india_states,
    color="scaled",
    hover_name="state_name",
    hover_data=["freq"],
    title="India Population Density",
    mapbox_style="carto-positron",
    center={"lat": 24, "lon": 78},
    zoom=3,
    opacity=0.5,
)
fig.show()

## USeless

In [ ]:
scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

fig = go.Figure(data=go.Scattergeo(
    lon=coords['long'],
    lat=coords['lat'],
    text=coords['freq'].astype(str)+" Mentions",
    geojson=india_states,
    marker = dict(
        color = coords['scaled'],
        colorscale = scl,
        reversescale = True,
        opacity = coords['scaled']/1.5,
        size = 20,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 0.1
        )
    )))

fig.update_layout(
    title="test",
    geo_scope="asia")
fig.show()

## DEcent

In [ ]:
fig = px.density_mapbox(coords, 
                        lat='lat', 
                        lon='long', 
                        z='freq', 
                        radius=20, 
                        center={"lat": 24, "lon": 78},
                        zoom=5,
                        mapbox_style="stamen-terrain")

fig.update_layout(title="Geographical mentions")
fig.show()

## EPIC

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=coords['lat'], 
                        lon=coords['long'], 
                        z=coords['freq'], 
                        radius=20))
fig.update_layout(mapbox_style="carto-darkmatter", mapbox_center_lon=78, mapbox_center_lat=24, mapbox_zoom=2.8)
fig.update_layout(margin={"r":100,"t":0,"l":50,"b":20})
fig.show()